In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px
import os
import warnings
import json
from datetime import datetime, date
import os
import shapefile
from datetime import datetime as dt
from sentinelhub import CRS, BBox, DataCollection, SentinelHubCatalog, SHConfig
from sentinelhub.aws import AwsDownloadClient
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import yaml
# open shape files for Cameroon
import geopandas as gpd


warnings.filterwarnings('ignore')

In [ ]:
# Open file for credentials
PATH = '/Users/noeliaotero/Documents/Geo_analysis/'
# Read YAML file
with open(PATH + "credentials.yml", 'r') as stream:
    credentials_conf = yaml.safe_load(stream)
with open(PATH + "config_sentinel.yml", 'r') as stream:
    s_conf = yaml.safe_load(stream)


In [ ]:

shafile_file = "/Users/noeliaotero/Documents/Geo_analysis/cmr/cmr.shp"
sh_cmr = gpd.read_file("/Users/noeliaotero/Documents/Geo_analysis/cmr/cmr.shp")

In [ ]:
#geojson_data = shapefile.Reader(shafile_file).__geo_interface__

In [ ]:
sh_cmr.plot()

In [ ]:


# connect to the API
api = SentinelAPI(credentials_conf['user'], credentials_conf['password'], 'https://apihub.copernicus.eu/apihub')

In [ ]:
footprint = geojson_to_wkt(read_geojson(s_conf['footprint_file']))

In [ ]:
s_conf['platformname']

In [ ]:
products = api.query(footprint,
                     date = (date(2020,1,1),date(2020,1,5)),
                     platformname = s_conf['platformname'],
                     processinglevel = s_conf['processinglevel'],
                     cloudcoverpercentage = (0, 20))
# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
images_df = api.to_geodataframe(products)

# GeoJSON FeatureCollection containing footprints and metadata of the scenes
# api.to_geojson(products)

In [ ]:
images_df_sorted = images_df.sort_values(['cloudcoverpercentage'], ascending=[True])
images_df_sorted

In [ ]:
# download all results from the search
# api.download_all(products)

In [ ]:
print(f"Number of products available: {len(images_df)}")
print(f"Downloading product: {images_df.head(s_conf['download_index'])['title']}")

# Select and download product to data directory.
images_df = images_df.head(s_conf['download_index'])  # Get desired product based on selected index.
cwd = os.getcwd()  # remember current work directory (CWD).
os.chdir(s_conf['data_dir'])  # Change directory.
api.download_all(images_df.index)  # Download product. Needs '.index' as it cannot download df directly.
os.chdir(cwd)  # Return to project directory.


In [ ]:
import zipfile

# -- Proprietary modules -- #


def unzip(sentinelsat_options: dict):
    # -- Unzips all files in data directory. -- #
    cwd = os.getcwd()
    os.chdir(sentinelsat_options['data_dir'])

    files = os.listdir()
    zip_files = [file for file in files if '.zip' in file]  # Find all .zip files.
    safe_files = [file for file in files if '.SAFE' in file]  # Find all .SAFe files.

    files_to_unzip = [file for file in zip_files if not
                      [file.split('.')[0] in safe_file for safe_file in safe_files]]  # Find .zip files not unzipped.

    # Unzip files.
    for file_to_unzip in files_to_unzip:
        to_unzip = zipfile.ZipFile(file_to_unzip, 'r')
        to_unzip.extractall()
        to_unzip.close()
    os.chdir(cwd)